<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=335e81378a01918ec539ddc003aa5d55e5fa725302008215630b8256d70958ca
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-21 09:50:14
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.44 C
-------------------
Today PnL: -70.12 K (-0.49%)
Current PnL: -21.91 L (-14.36%)
CY Booked + Current PnL: -8.02 L (-5.26%)
-------------------
Total profit:  1.68 L
Total loss:  -23.59 L
-------------------
Total Booked + Current PnL: 18.96 L (15.09%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.65%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.32 L (60.69%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.25,-14.01,16.40,0.09,14212.0,-14121.0,86656.0,101.07,50.0,M-SC,4.16,253.0,-0.99,0.60,13.85,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.31,-8.38,9.16,0.02,14650.0,-14624.0,159935.0,143.86,73.0,M-SC,4.25,234.0,-1.00,1.11,39.12,OX40N,NTT,PAINTS
83,VOLTAS,1530.00,-0.38,10.03,8.78,19.69,18524.0,19233.0,210975.0,-0.66,57.0,X-MC,3.59,78.0,1.04,1.47,17.51,XY25,NTT,AC
43,ITC,452.00,-0.11,-1.05,11.60,10.44,22973.0,-2093.0,198045.0,-40.74,44.0,X-LC,0.98,5.0,-0.09,1.38,4.46,X40,NTT,FMCG
35,ICICIGI,2252.93,-1.60,6.97,12.37,20.19,23066.0,12145.0,186465.0,-16.25,51.0,X-MC,7.66,68.0,0.53,1.30,23.13,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIS,528.00,2.41,-24.00,59.56,21.26,50257.0,-26652.0,84380.0,1990.67,45.0,H-SC,3.24,166.0,-0.53,0.59,14.01,OX40N,NTT,MISC
68,SONACOMS,806.63,1.53,-10.84,56.51,39.54,50969.0,-10964.0,90195.0,-30.23,73.0,M-SC,6.95,220.0,-0.22,0.63,27.18,AR,ATH,AUTO
50,LTIM,7201.88,0.92,10.18,19.49,31.66,52860.0,25064.0,271215.0,6.01,69.0,H-LC,11.85,43.0,0.47,1.89,50.48,X200,ATH,IT
52,MEDANTA,1486.00,0.91,2.22,21.39,24.08,27757.0,2815.0,129765.0,-6.13,43.0,X-SC,4.52,89.0,0.10,0.90,22.27,XY24,NTT,HEALTHCARE
0,5PAISA,593.00,0.88,-31.51,64.84,12.90,97037.0,-68840.0,149656.0,119.95,69.0,H-SC,14.88,161.0,-0.71,1.04,24.44,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-2.62,-14.71,79.96,53.50,205795.0,-44379.0,257372.0,-58.65,48.0,X-MC,3.85,58.0,-0.22,1.79,14.32,XY24,NTT,FMCG
56,QUESS,424.00,-1.85,-28.65,100.19,42.84,46472.0,-18622.0,46384.0,-53.17,37.0,X-SC,37.14,83.0,-0.40,0.32,0.50,XY24,NTT,MISC
13,BSOFT,831.70,-1.70,-30.71,117.61,50.79,114624.0,-43192.0,97461.0,-11.09,54.0,H-SC,7.66,151.0,-0.38,0.68,13.48,XR,ATH,IT
35,ICICIGI,2252.93,-1.60,6.97,12.37,20.19,23066.0,12145.0,186465.0,-16.25,51.0,X-MC,7.66,68.0,0.53,1.30,23.13,X40,ATH,INSURANCE
57,RAJESHEXPO,518.00,-1.58,-63.66,175.36,0.07,87420.0,-87325.0,49852.0,1727.02,57.0,L-SC,2.04,268.0,-1.00,0.35,23.98,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.78,0.22,71.88,72.27,122069.0,375.0,169823.0,-13.79,52.0,M-LC,5.06,99.0,0.0,1.18,6.41,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.89,-3.67,115.82,107.91,164733.0,-5412.0,142232.0,-21.77,36.0,M-SC,10.19,216.0,-0.03,0.99,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.31,-8.38,9.16,0.02,14650.0,-14624.0,159935.0,143.86,73.0,M-SC,4.25,234.0,-1.00,1.11,39.12,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.25,-14.01,16.40,0.09,14212.0,-14121.0,86656.0,101.07,50.0,M-SC,4.16,253.0,-0.99,0.60,13.85,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.27,-20.99,43.62,13.47,92939.0,-56601.0,213066.0,-67.53,31.0,H-SC,1.67,158.0,-0.61,1.48,8.34,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,-0.71,-23.21,71.92,32.03,72813.0,-30593.0,101242.0,-16.88,47.0,H-SC,6.50,148.0,-0.42,0.70,13.40,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.78,0.22,71.88,72.27,122069.0,375.0,169823.0,-13.79,52.0,M-LC,5.06,99.0,0.00,1.18,6.41,XR,NTT,IT
38,INDIAMART,4810.62,-1.48,1.26,104.16,106.72,130079.0,1548.0,124884.0,-49.87,37.0,H-SC,4.45,139.0,0.01,0.87,24.93,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.48,1.26,104.16,106.72,130079.0,1548.0,124884.0,-49.87,37.0,H-SC,4.45,139.0,0.01,0.87,24.93,AR,ATH,MISC
85,WIPRO,420.00,-0.78,0.22,71.88,72.27,122069.0,375.0,169823.0,-13.79,52.0,M-LC,5.06,99.0,0.00,1.18,6.41,XR,NTT,IT
25,FINCABLES,1641.55,-0.89,-3.67,115.82,107.91,164733.0,-5412.0,142232.0,-21.77,36.0,M-SC,10.19,216.0,-0.03,0.99,0.00,OX40N,ATH,CABLES
37,IEX,219.00,0.09,-2.78,52.87,48.63,102781.0,-5550.0,194404.0,-35.29,58.0,H-SC,15.35,136.0,-0.05,1.35,8.46,XR,NTT,MISC
84,WHIRLPOOL,2270.00,-0.08,-4.87,91.24,81.92,99638.0,-5594.0,109204.0,-44.29,32.0,M-SC,6.20,236.0,-0.06,0.76,29.28,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.12,-34.96,107.90,35.20,90292.0,-44989.0,83681.0,9.75,23.0,X-SC,12.06,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,600.00,0.67,-14.17,65.65,42.18,155749.0,-39169.0,237241.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.67,XY24,NTT,AUTO
34,HONAUT,58357.33,-0.10,-14.90,68.42,43.32,94830.0,-24272.0,138600.0,-26.83,26.0,X-SC,1.66,90.0,-0.26,0.96,6.42,X40N,ATH,ELECTRICAL
46,JSWINFRA,342.00,-0.60,-7.48,24.54,15.23,45486.0,-14992.0,185355.0,-20.80,28.0,X-MC,6.33,66.0,-0.33,1.29,22.35,X40N,NTT,REALTY
9,BAJAJHFL,181.50,-0.79,-13.65,71.92,48.45,124519.0,-27371.0,173135.0,-19.87,29.0,X-MC,7.29,64.0,-0.22,1.20,0.00,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.67,-14.17,65.65,42.18,155749.0,-39169.0,237241.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.67,XY24,NTT,AUTO
43,ITC,452.00,-0.11,-1.05,11.60,10.44,22973.0,-2093.0,198045.0,-40.74,44.0,X-LC,0.98,5.0,-0.09,1.38,4.46,X40,NTT,FMCG
36,ICICIPRULI,790.00,-0.86,1.63,28.86,30.97,51839.0,2886.0,179624.0,-22.29,50.0,X-MC,1.50,75.0,0.06,1.25,14.40,X40,ATH,INSURANCE
34,HONAUT,58357.33,-0.10,-14.90,68.42,43.32,94830.0,-24272.0,138600.0,-26.83,26.0,X-SC,1.66,90.0,-0.26,0.96,6.42,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.48,-22.57,111.99,64.15,206342.0,-53701.0,184250.0,-54.57,48.0,X-SC,1.81,82.0,-0.26,1.28,3.35,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.12,-34.96,107.90,35.20,90292.0,-44989.0,83681.0,9.75,23.0,X-SC,12.06,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.79,-13.65,71.92,48.45,124519.0,-27371.0,173135.0,-19.87,29.0,X-MC,7.29,64.0,-0.22,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,0.17,-5.63,33.59,26.07,51903.0,-9220.0,154520.0,-30.55,32.0,X-MC,8.71,55.0,-0.18,1.07,0.17,X40,ATH,APPARELS
56,QUESS,424.00,-1.85,-28.65,100.19,42.84,46472.0,-18622.0,46384.0,-53.17,37.0,X-SC,37.14,83.0,-0.40,0.32,0.50,XY24,NTT,MISC
76,TMPV,600.00,0.67,-14.17,65.65,42.18,155749.0,-39169.0,237241.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.67,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.85,-28.65,100.19,42.84,46472.0,-18622.0,46384.0,-53.17,37.0,X-SC,37.14,83.0,-0.40,0.32,0.50,XY24,NTT,MISC
60,RELAXO,1176.00,-0.59,-46.67,185.58,52.31,143672.0,-67742.0,77418.0,-43.18,31.0,X-SC,3.41,91.0,-0.47,0.54,3.63,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.12,-34.96,107.90,35.20,90292.0,-44989.0,83681.0,9.75,23.0,X-SC,12.06,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.91,2.22,21.39,24.08,27757.0,2815.0,129765.0,-6.13,43.0,X-SC,4.52,89.0,0.10,0.90,22.27,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-0.70,-2.34,24.50,21.58,33529.0,-3285.0,136854.0,-50.76,39.0,X-MC,7.17,56.0,-0.10,0.95,21.16,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.11,-14.46,39.44,19.28,116719.0,-50016.0,295940.0,-25.57,63.0,X-LC,7.77,1.0,-0.43,2.06,9.40,X40,ATH,IT
41,INFY,2275.00,-0.30,4.60,48.06,54.87,157288.0,14386.0,327274.0,-18.52,58.0,X-LC,3.42,2.0,0.09,2.27,13.26,X40,BTT,IT
76,TMPV,600.00,0.67,-14.17,65.65,42.18,155749.0,-39169.0,237241.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.67,XY24,NTT,AUTO
82,VBL,671.64,-0.82,-9.54,49.99,35.67,142819.0,-30146.0,285696.0,-20.30,41.0,X-LC,6.13,4.0,-0.21,1.99,2.99,X40N,ATH,FMCG
43,ITC,452.00,-0.11,-1.05,11.60,10.44,22973.0,-2093.0,198045.0,-40.74,44.0,X-LC,0.98,5.0,-0.09,1.38,4.46,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.78,-11.97,106.39,81.67,87854.0,-11233.0,82577.0,7427.78,54.0,L-SC,18.44,271.0,-0.13,0.57,59.91,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.53,-10.84,56.51,39.54,50969.0,-10964.0,90195.0,-30.23,73.0,M-SC,6.95,220.0,-0.22,0.63,27.18,AR,ATH,AUTO
51,MASFIN,398.61,-1.05,-3.55,26.54,22.05,25080.0,-3480.0,94500.0,-16.89,55.0,H-SC,7.02,164.0,-0.14,0.66,37.36,XR,ATH,FINANCE
65,SHALBY,327.00,-1.27,-16.06,49.04,25.10,74562.0,-29099.0,152044.0,1018.02,33.0,M-SC,16.66,250.0,-0.39,1.06,21.85,XY24,NTT,HEALTHCARE
39,INDIGOPNTS,1408.00,-1.31,-8.38,9.16,0.02,14650.0,-14624.0,159935.0,143.86,73.0,M-SC,4.25,234.0,-1.00,1.11,39.12,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-1.31,-8.38,9.16,0.02,14650.0,-14624.0,159935.0,143.86,73.0,M-SC,4.25,234.0,-1.00,1.11,39.12,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.30,0.25,20.64,20.95,52097.0,642.0,252410.0,-0.77,76.0,X-LC,12.10,31.0,0.01,1.75,35.98,X40,ATH,PAINTS
0,5PAISA,593.00,0.88,-31.51,64.84,12.90,97037.0,-68840.0,149656.0,119.95,69.0,H-SC,14.88,161.0,-0.71,1.04,24.44,OX40N,NTT,FINANCE
68,SONACOMS,806.63,1.53,-10.84,56.51,39.54,50969.0,-10964.0,90195.0,-30.23,73.0,M-SC,6.95,220.0,-0.22,0.63,27.18,AR,ATH,AUTO
40,INDUSINDBK,1800.00,0.66,-38.37,115.61,32.89,55014.0,-29620.0,47586.0,-687.51,61.0,L-MC,5.33,259.0,-0.54,0.33,31.05,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.56
2,50,75.38


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.71
MC    29.25
LC    25.04
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.92
X40      21.06
X40N     11.87
XR        9.60
XY25      9.09
AR        9.03
OX40N     7.74
X200      1.89
SR        1.00
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.10
X-MC    22.83
X-LC    19.89
M-SC    12.05
X-SC     8.15
H-MC     4.71
H-LC     3.00
L-SC     1.41
M-MC     1.38
M-LC     1.18
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.70,-4.72,38.58
IT,13.79,-13.93,72.75
FINANCE,9.47,-13.75,61.81
MISC,7.48,-20.64,70.07
PAINTS,6.01,-6.72,22.57
ELECTRICAL,5.88,-10.97,51.62
INSURANCE,4.44,0.98,33.92
PHARMA,3.85,-1.01,33.34
AUTO,3.36,-18.96,69.97


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3018429.0,21
XR,1282125.0,13
AR,1274216.0,10
X40,1007610.0,14
X40N,741643.0,9
OX40N,676910.0,10
XY25,332869.0,6
SR,276992.0,2
MH,68758.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3510956.0,25
M-SC,1325597.0,15
X-MC,1198968.0,16
X-LC,900884.0,11
X-SC,745465.0,8
H-MC,401847.0,3
L-SC,258795.0,3
H-LC,126485.0,2
M-LC,122069.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1207035.0      6
           AR         864266.0      5
           XR         785840.0      7
M-SC       XY24       775538.0      6
X-MC       X40        437519.0      7
X-LC       X40        394456.0      5
X-MC       XY24       344237.0      3
H-SC       OX40N      308065.0      4
X-SC       X40N       289259.0      3
M-SC       OX40N      281425.0      5
X-SC       XY24       280571.0      3
H-SC       SR         276992.0      2
X-MC       X40N       243707.0      4
X-LC       XY24       224719.0      2
H-MC       AR         215518.0      2
X-LC       X40N       208677.0      2
H-MC       XY24       186329.0      1
X-SC       X40        175635.0      2
X-MC       XY25       173505.0      2
L-SC       XR         171375.0      2
M-SC       XR         147827.0      2
M-LC       XR         122069.0      1
M-SC       AR         120807.0      2
L-SC       OX40N       87420.0      1
H-LC       AR          73625.0      1
X-LC       XY25        73032.0      2
H-SC       MH          68758.0      1
L-MC       XR          55014.0      1
H-LC       X200        52860.0      1
M-MC       XY25        47157.0      1
L-LC       XY25        39175.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
